# Introduction

The purpose of this notebook is to train an image classification model for the Hot Dog Not Hot Dog dataset. The notebook consists of the following steps:

 - Using kaggle API for downloading
 - Applying Data Augmentation with `ImageDataGenerator`
 - Applying Transfer Learning for the VGG19 model
 - Restoring a model with the lowest validation loss
 - Applying Early stopping [optional]



## Apply kaggle API

We'll use kaggle API in order to download the Hot Dog - Not Hot Dog dataset. [Link to dataset](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog). We chose to use kaggle because this is a reliable source. Usually, many links for downloading datasets are outdated. On kaggle, datasets stay for years. 

In order to use kaggle API you need a kaggle account with a valid API key. All the steps are explained under [Easiest way to download kaggle data in Google Colab](https://www.kaggle.com/general/74235)

In [1]:
! pip install -q kaggle

Please uncomment the following cell. I commented it because my API Key would be visible in the output of the cell. I wanted to hide it.

In [2]:
# from google.colab import files
# files.upload()

In [3]:
# create a dir for kaggle API
! mkdir ~/.kaggle
# copy the json file with API Key
! cp kaggle.json ~/.kaggle
# make it read-only
! chmod 600 ~/.kaggle/kaggle.json
# download dataset and unzip it locally
! kaggle datasets download -d dansbecker/hot-dog-not-hot-dog -p /content/sample_data/ --unzip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 92% 82.0M/89.3M [00:00<00:00, 68.3MB/s]
100% 89.3M/89.3M [00:00<00:00, 95.6MB/s]


# Code

## Imports

In [4]:
from tensorflow import keras
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

## Data Augmentation

The dataset is relatively small. It contains only 498 images in the train set. Applying Data Augmentation is a very convienient way to artifficially increase the number of train images.

In [5]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [6]:
# Define the batch size:
batch_size=32

# Define the train and validation generators: 
train_generator = train_datagen.flow_from_directory(
    directory='/content/sample_data/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='/content/sample_data/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


## Downloading VGG19

In [7]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

### Freezing layers

In [8]:
for layer in vgg19.layers:
    layer.trainable = False

## Implementing a Keras Sequential Model

In [16]:
# Instantiate the sequential model and add the VGG19 model: 
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model: 
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

## Training the Model

In [17]:
# apply early stoping if no improvement for at least 5 steps (patience)
# set restore to True
early_stoping_callback=keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=2, mode='auto',
    baseline=None, restore_best_weights=True)

checkpoint_filepath = '/content/sample_data/weights.{epoch:02d}.h5'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss', #  validation loss
    mode='min',         # lowest possible
    save_best_only=True)

# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train model
history1 = model.fit(train_generator, steps_per_epoch=15, 
                    epochs=15, validation_data=valid_generator, 
                    validation_steps=15, 
                    callbacks=[
                            #    early_stoping_callback, # early stopping 
                               model_checkpoint_callback # only best model 
                               ]
                )

Epoch 1/15
15/15 [==============================] - 14s 888ms/step - loss: 0.8617 - accuracy: 0.5987 - val_loss: 0.8252 - val_accuracy: 0.5896
Epoch 2/15
15/15 [==============================] - 13s 859ms/step - loss: 0.6995 - accuracy: 0.6760 - val_loss: 0.4640 - val_accuracy: 0.7729
Epoch 3/15
15/15 [==============================] - 13s 865ms/step - loss: 0.4497 - accuracy: 0.7940 - val_loss: 0.4579 - val_accuracy: 0.7812
Epoch 4/15
15/15 [==============================] - 13s 864ms/step - loss: 0.4103 - accuracy: 0.8090 - val_loss: 0.6384 - val_accuracy: 0.7021
Epoch 5/15
15/15 [==============================] - 13s 854ms/step - loss: 0.3905 - accuracy: 0.8262 - val_loss: 0.6002 - val_accuracy: 0.7292
Epoch 6/15
15/15 [==============================] - 13s 855ms/step - loss: 0.3651 - accuracy: 0.8369 - val_loss: 0.4716 - val_accuracy: 0.7750
Epoch 7/15
15/15 [==============================] - 13s 849ms/step - loss: 0.3429 - accuracy: 0.8455 - val_loss: 0.4569 - val_accuracy: 0.7875

## Evaluating the Models

### Model after the last epoch

In [18]:
score = model.evaluate(valid_generator, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

Test loss: 0.626677930355072
Test accuracy: 0.7599999904632568


### The best Model
However, the 7th epoch had the lowest validation loss. Let's load the model and compare both models

In [20]:
best_model = keras.models.load_model('/content/sample_data/weights.07.h5')
best_score = best_model.evaluate(valid_generator, verbose=0)
print(f"Test loss: {best_score[0]}")
print(f"Test accuracy: {best_score[1]}")

Test loss: 0.455517053604126
Test accuracy: 0.7879999876022339


The model that we had after 7th epoch performs better. This is our baseline model for now.

In [ ]:
# best_model.save('./models/hotdog_vgg.h5')

# Final Thoughts

In this notebook we showed how to apply transfer learning using a VGG19 model. We achieved around 80% accuracy. It may not seem impressive. Yet, the Hot Dog Dataset is quite small. There are many images that are tricky. If an image contains only a hot dog, our model classifies it correctly with a high probability. Probably, the model could increase accuracy by applying Fine-tuning. It is out of the scope of this notebook.

These steps probably won't bring any value:

- we could increase the number of epochs. However, it's obvious the model starts to overfit. The difference between train and validation metrics increases.
- applying early stopping when there are only 15 epochs isn't beneficial.

Further exploration:

- compare performance without Data Augmentation
- apply Fine-tuning